Modelo LSTM
3 colunas
pytorch
bayesian optimization

In [185]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import os
from datetime import datetime


In [186]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

Using cuda


In [187]:
# cria pasta de logs com data de hoje

data_hoje = datetime.now().strftime('%d-%m')
os.makedirs(f'../logs/{data_hoje}', exist_ok=True)
os.makedirs(f'../plots/{data_hoje}', exist_ok=True)

In [188]:
import logging

inicio_execucao = pd.Timestamp.now()

logging.basicConfig(filename=f'../logs/{data_hoje}/lstm.log', level=logging.INFO, format='- %(message)s')
logging.info('-' * 50)
logging.info(f'{inicio_execucao} - Iniciando o processo de treinamento do modelo LSTM')

In [189]:
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono'], low_memory=False)

In [190]:
df_original['Data e Hora'] = pd.to_datetime(df_original['Data e Hora'], format='%Y-%m-%d %H:%M:%S')
df_original.index = df_original['Data e Hora']
df_original.sort_index(inplace=True)

colunas_selecionadas = ['PM2.5', 'PM10', 'Monóxido de Carbono']
df = df_original[colunas_selecionadas]

df = df.loc['2019-01-01':'2022-01-01']

df = df.apply(pd.to_numeric, errors='coerce')

logging.info(f'Colunas Selecionadas: {colunas_selecionadas}')
df.head(20)

,PM2.5,PM10,Monóxido de Carbono
Data e Hora,,,
2019-01-01 00:30:00,37.0,45.12,0.77
2019-01-01 01:30:00,23.0,70.53,0.92
2019-01-01 02:30:00,18.0,68.99,0.81
2019-01-01 03:30:00,13.0,59.54,0.57
2019-01-01 04:30:00,7.0,30.84,0.44
2019-01-01 05:30:00,2.0,17.32,0.43
2019-01-01 06:30:00,NaN,8.84,0.40
2019-01-01 07:30:00,NaN,16.81,0.41
2019-01-01 08:30:00,NaN,9.08,0.41


In [191]:
def impute_missing_values(df):
    return df.interpolate(method='linear', limit_direction='both')


df_imputed = impute_missing_values(df)

logging.info(f"Dados ausentes antes da imputação: {df.isna().sum()}")
logging.info(f"Dados ausentes após a imputação: {df_imputed.isna().sum()}")

In [192]:
from copy import deepcopy as dc
from sklearn.preprocessing import MinMaxScaler

# Preparando os dados para LSTM
def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)
    for col in colunas_selecionadas:
        for i in range(1, n_steps + 1):
            df[f'{col}(t-{i})'] = df[col].shift(i)
    df.dropna(inplace=True)
    return df


lookback = 8  # 8 horas de lookback
shifted_df = prepare_dataframe_for_lstm(df_imputed, lookback)

# Normalizando os dados
scaler = MinMaxScaler(feature_range=(0, 1))
shifted_df_as_np = scaler.fit_transform(shifted_df)

X = shifted_df_as_np[:, len(colunas_selecionadas):]
y = shifted_df_as_np[:, 0]  # Mantemos PM2.5 como nossa variável alvo

X = dc(np.flip(X, axis=1))

# Dividindo em conjuntos de treino, validação e teste
train_split = int(len(X) * 0.7)
val_split = int(len(X) * 0.85)

X_train, X_val, X_test = X[:train_split], X[train_split:val_split], X[val_split:]
y_train, y_val, y_test = y[:train_split], y[train_split:val_split], y[val_split:]


In [193]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        batch_size = x.size(0)
        
        # Inicialize hidden state com zeros
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        # Inicialize cell state
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decodifique o último estado oculto
        out = self.fc(out[:, -1, :])
        return out

In [194]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch.unsqueeze(1))
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        
        if (epoch + 1) % 10 == 0:
            logging.info(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    return train_losses, val_losses

In [195]:
# Definindo o espaço de busca para o otimizador bayesiano
search_space = {
    'hidden_size': Integer(32, 256),
    'num_layers': Integer(1, 3),
    'dropout': Real(0.0, 0.5),
    'learning_rate': Real(1e-4, 1e-2, prior='log-uniform'),
    'batch_size': Categorical([32, 64, 128])
}


In [196]:
def objective(**params):
    hidden_size = int(params['hidden_size'])
    num_layers = int(params['num_layers'])
    dropout = params['dropout']
    learning_rate = params['learning_rate']
    batch_size = int(params['batch_size'])
    
    model = LSTM(input_size=X_train.shape[1], hidden_size=hidden_size, num_layers=num_layers, 
                 output_size=1, dropout=dropout).to(device)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Reformatar os dados para (batch_size, sequence_length, input_size)
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_val_reshaped = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train_reshaped), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val_reshaped), torch.FloatTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    _, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)
    
    return -val_losses[-1]  # Retorna o negativo da perda de validação final (queremos maximizar)

# Definindo o espaço de busca para o otimizador bayesiano
pbounds = {
    'hidden_size': (32, 256),
    'num_layers': (1, 3),
    'dropout': (0.0, 0.5),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': (32, 128)
}


In [197]:
from bayes_opt import BayesianOptimization
# Executando a otimização bayesiana
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=5,
    n_iter=45,
)

# Registrando os melhores parâmetros encontrados
best_params = optimizer.max['params']
logging.info(f"Melhores parâmetros encontrados: {best_params}")
logging.info(f"Melhor pontuação: {-optimizer.max['target']}")  # Note o sinal negativo aqui

# Ajustando os tipos de dados dos melhores parâmetros
best_params['hidden_size'] = int(best_params['hidden_size'])
best_params['num_layers'] = int(best_params['num_layers'])
best_params['batch_size'] = int(best_params['batch_size'])

# Treinando o modelo final com os melhores parâmetros
final_model = LSTM(input_size=X_train.shape[1], hidden_size=best_params['hidden_size'], 
                   num_layers=best_params['num_layers'], output_size=1, 
                   dropout=best_params['dropout']).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))

train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'])

train_losses, val_losses = train_model(final_model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

|   iter    |  target   | batch_... |  dropout  | hidden... | learni... | num_la... |
-------------------------------------------------------------------------------------


C:\dev\scripts\pm25-plots\venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.36016224672107905 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


| 1         | -0.001305 | 72.03     | 0.3602    | 32.03     | 0.003093  | 1.294     |
| 2         | -0.001301 | 40.86     | 0.09313   | 109.4     | 0.004028  | 2.078     |


C:\dev\scripts\pm25-plots\venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.34260975019837975 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


| 3         | -0.001334 | 72.24     | 0.3426    | 77.8      | 0.008793  | 1.055     |


C:\dev\scripts\pm25-plots\venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.20865240118356349 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


| 4         | -0.001274 | 96.36     | 0.2087    | 157.1     | 0.00149   | 1.396     |
| 5         | -0.001311 | 108.9     | 0.4841    | 102.2     | 0.006954  | 2.753     |
| 6         | -0.001319 | 97.13     | 0.3437    | 157.8     | 0.003645  | 2.226     |
| 7         | -0.001297 | 114.7     | 0.1895    | 122.3     | 0.003854  | 2.007     |
| 8         | -0.001262 | 73.12     | 0.2821    | 242.0     | 0.00446   | 2.438     |
| 9         | -0.001271 | 73.12     | 0.2838    | 242.0     | 0.006101  | 2.44      |
| 10        | -0.001284 | 73.11     | 0.2765    | 242.0     | 0.0001    | 2.433     |
| 11        | -0.00128  | 73.12     | 0.2834    | 242.0     | 0.0001    | 2.44      |
| 12        | -0.001264 | 73.11     | 0.2793    | 242.0     | 0.01      | 2.435     |
| 13        | -0.001316 | 120.2     | 0.3022    | 144.0     | 0.004184  | 2.862     |
| 14        | -0.001311 | 32.48     | 0.1346    | 156.6     | 0.009566  | 2.806     |
| 15        | -0.001268 | 73.1      | 0.2605    | 242.

KeyboardInterrupt: 